<a href="https://colab.research.google.com/github/AlvarDev/Gemini-is-all-you-need/blob/main/Gemini_is_all_you_need.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Based on https://github.com/GoogleCloudPlatform/generative-ai/tree/main

# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini é tudo o que você precisa

[Add Shorcuts to open by their own]

## Overview

Este notebook mostra diferente cénarios de como o Gemini pode ser utilizado no dia a dia dos desenvolvedores.

A temática é de futebol.

Este notebook usa exemplos de outros notebooks no [GitHub do Google Cloud](https://github.com/GoogleCloudPlatform/generative-ai/tree/main).



## Custos

Este tutorial usa componentes faturáveis ​​do Google Cloud:

* Vertex AI
* BigQuery

Saiba mais sobre [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing) e [BigQuery](https://cloud.google.com/bigquery/pricing).

Use a [calculadora de preços](https://cloud.google.com/products/calculator/)
para gerar uma estimativa de custo com base no uso projetado.

## Começamos

### Instalar o SDK da Vertex AI e BigQuery para Python


In [ ]:
! pip3 install --upgrade --user google-cloud-aiplatform google-cloud-bigquery

### Reinicie o Runtime
Para usar os pacotes recém-instalados neste runtime do Jupyter, você deve reiniciar o runtime. Você pode fazer isso executando a seguinte célula, que irá reiniciar o kernel atual.


In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython
import time

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Authenticate your notebook environment (Colab only)
Se você estiver executando este notebook no Google Colab, execute a célula a seguir para autenticar seu ambiente. Esta etapa não é necessária se você estiver usando o [Vertex AI Workbench](https://cloud.google.com/vertex-ai-notebooks?hl=en).

In [ ]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Defina as informações do projeto do Google Cloud e inicialize o Vertex AI
Inicialize o SDK da Vertex AI para Python para seu projeto:

In [ ]:
# Define project information
PROJECT_ID = "sre-demos"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

BQ_PROJECT_ID = "sre-demos"  # @param {type:"string"}
BQ_LINKED_DATASET = "fifa23"  # @param {type:"string"}

# Initialize Vertex AI and BigQuery
from google.cloud import bigquery
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)
client = bigquery.Client(project=BQ_PROJECT_ID)

### Import libraries

In [ ]:
from IPython.display import Markdown
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Image,
    Part,
)

## Clube de Regatas Ganondorf - usando texto com Gemini

**Quem torce pelo Clube de Regatas Ganondorf?**

*Clube de Regatas Ganondorf é um FIFA Ultimate Team criado por Alvaro e Luciano. Alvaro é o técnico do time e Luciano atua como presidente do time, Luciano decide os novos contratos dos jogadores e quem sai do time. O capitão da equipe é Marc-André ter Stegen e os jogadores mais populares são Evanilson, Thierry Henry, Luka Modrić e Ronaldo. A equipe CR Ganondorf venceu a Liga dos Campeões da UEFA em 2020, 2022 e 2023.*


Essa informação não é 'pública' pois é informação gerada em casa, se perguntamos para o Gemini sobre o **CR Ganondorf** provavelmente a resposta vai ser muito diferente.

### Para esta primeira parte vamos o usar o Gemini 1.0 Pro

In [ ]:
model = GenerativeModel("gemini-1.5-pro-preview-0409")

Construimos o promt

In [ ]:
prompt = "O que é o Clube de Regatas Ganondorf?"

contents = [
    prompt,
]

response = model.generate_content(contents)

print("\n-------Response--------")
Markdown(response.text)

A resposta (no momento que criei a demo):

"*O Clube de Regatas Ganondorf **não existe**. Ganondorf é um personagem fictício da série de videogames The Legend of Zelda.*"

Para indicar ao Gemini a respota desejada, temos que forneces um contexto no promt, vamos usar o conceito que vimos ao inicio.

Adicionalmente podemos indicar ao Gemini responder num formato especifico, vamos testar com JSON:

In [ ]:
# vars names are not pre-defined, you can use your own vars

context = """
            Clube de Regatas Ganondorf é um FIFA Ultimate Team criado por Alvaro e Luciano.
            Alvaro é o técnico do time e Luciano atua como presidente do time,
            Luciano decide os novos contratos dos jogadores e quem sai do time.
            O capitão da equipe é Marc-André ter Stegen e os jogadores mais populares são
            Evanilson, Thierry Henry, Luka Modrić e Ronaldo.
            A equipe CR Ganondorf venceu a Liga dos Campeões da UEFA em 2020, 2022 e 2023.
          """

questions = """
              Responde as seguintes perguntas:
              O que é o Clube de Regatas Ganondorf?
              Quem é o capitão do time?
              Quem é o presidente do clube?
            """

format =  """
            Em formato JSON:
            {
              "question": "pergunta",
              "answer": "resposta"
            }
          """

contents = [
    context,
    questions,
    format,
]

response = model.generate_content(contents)

print("\n-------Response--------")
Markdown(response.text)

## Querying Fifa 2023 Dataset no BigQuery

Neste exemplo vamos usar o dataset [**Fifa 2023 EDA**](https://www.kaggle.com/datasets/anshulchauhan21/fifa-2023-dataset/data) do [Kaggle](https://www.kaggle.com).

> O dataset FIFA 2023 é uma coleção abrangente de dados relacionados à popular franquia de videogame de futebol. Inclui informações sobre vários aspectos do jogo, como atributos do jogador, detalhes da equipe, classificações e muito mais.

Vamos usar o [BigQuery](https://cloud.google.com/bigquery?hl=en) como engine de banco de dados, porém este exemplo pode ser aplicado a diferentes engines de bancos de dados.

Criar o dataset no BigQuery não faz parte deste notebook, segue o link para o tutorial [Criar FIFA 23 Dataset](https://drive.google.com/file/d/19ARkbocNItJqE65CVh3tV2GskV1dCt7I/view).






### Criando o prompt

Para construir este prompt precisamos das seguintes partes:

*   **instructions**: As indicações inicias que o Gemini deve seguir.
*   **schema_prompt**: Indicação que apresenta o schema da tabela.
*   **few_examples**: Alguns exemplos de como fazer as queries.
*   **question**: Pergunta enviada pelo usuario.
*   **answer**: Resposta do Gemini.

#### Instructions

In [ ]:
instructions = """
  Esta é uma tarefa que converte texto em instrução GoogleSQL.
  Primeiro forneceremos o esquema do conjunto de dados e, em seguida, faremos uma pergunta em texto.
  Você será solicitado a gerar uma instrução SQL válida para o BigQuery.
  Remova quaisquer delimitadores em torno da resposta, como "```"
  """

#### Schema Definition

![FIFA 23 Dataset schema definition](https://storage.googleapis.com/sre-demos-files/FIFA%2023%20dataset%20schema%20definition.png)


In [ ]:
BQ_TABLES=['players']
QUERY = f"""\
SELECT
    '[Schema (values)]: ' || '| log_summary | ' || STRING_AGG(table_values, ' | ') || ';' AS tables_definition,
    '[Column names (type)]: ' || STRING_AGG(column_names_types) || ';' AS columns_definition
FROM (
    SELECT
      table_name,
      table_name || ' : ' || STRING_AGG(column_name, ' , ') as table_values,
      STRING_AGG(table_name || ' : ' || column_name || ' (' || data_type || ')', ' | ') as column_names_types
    FROM {BQ_PROJECT_ID}.{BQ_LINKED_DATASET}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS
    WHERE table_name IN {'(' + ",".join(map(lambda x: f"'{x}'", BQ_TABLES)) + ')'}
    GROUP BY table_name
    ORDER BY table_name
)
"""

# Create query job
query_job = client.query(QUERY)
# Get first row
schema = next(query_job.result())

# Build schema definition
schema_prompt = f"""
  Definição de esquema de tabelas do BigQuery:
  {schema.tables_definition}

  {schema.columns_definition}
  """



print(schema_prompt)

#### Few Examples

Criamos alguns exemplos de como fazer a query

In [ ]:
few_examples = """
  Aqui estão alguns exemplos:

  Pergunta: Lista o ID, nome, numero de camisa, idade, nacionalidade e potencial  dos jogadores do Real Madrid CF?

  Resposta: select ID, Name as nome , Kit_Number as numero_de_camisa, age as idade, Nationality as nacionalidade, Potential as potencial  from `players` where Club like "%Real Madrid%"

  Pergunta: Quem é o jogador com maior potencial do Boca Juniors FC?

  Resposta: select ID, Name as nome , Kit_Number as numero_de_camisa, age as idade, Nationality as nacionalidade, Potential as potencial from `players` where Club like "%Boca Juniors%" order by Potential DESC Limit 1

  Pergunta: Qual era a meia de idades dos jogadores do Boca Juniors FC?

  Resposta: select AVG(age) as idade from `players` where Club like "%Boca Juniors%"
"""

print(few_examples)

### Gerar consultas SQL

#### Definir função auxiliar para gerar SQL


A seguinte função auxiliar `generate_sql()` é usada para recuperar uma consulta SQL do modelo Vertex AI LLM usando o modelo de prompt que construímos até agora.

Observe como `generate_sql()` usa a função `sanitize_output()` para reduzir a resposta à própria consulta SQL antes de retornar os resultados. Mesmo que o prompt do modelo inclua instruções para ajustar a saída do modelo, ainda pode haver aspas ou crases de bloco de código que precisam ser removidos para evitar um erro de sintaxe SQL subsequente.

In [ ]:
import re


# Strip text to include only the SQL code block with
def sanitize_output(text: str) -> str:
    # Strip whitespace and any potential backticks enclosing the code block
    text = text.strip()
    regex = re.compile(r"^\s*```(\w+)?|```\s*$")
    text = regex.sub("", text).strip()

    # Find and remove any trailing quote without corresponding opening quote
    if re.search(r'^[^"]*"$', text):
        text = text[:-1]
    # Find and remove any leading quote without corresponding closing quote
    if re.search(r'^"[^"]*$', text):
        text = text[1:]

    return text


# Call model using prompt and pre-defined parameters
def generate_sql(
    model,
    contents,
    temperature: float = 0.2,
    max_output_tokens: int = 1024,
    top_k: int = 40,
    top_p: float = 0.8,
) -> str:
    print("Generating SQL...")
    print("Number of input tokens: " + str(len(prompt)))

    # response = model.predict(
    #     prompt,
    #     temperature=temperature,
    #     max_output_tokens=max_output_tokens,
    #     top_k=top_k,
    #     top_p=top_p,
    # )

    response = model.generate_content(contents)

    text = response.text
    print("Number of output tokens: " + str(len(text)))
    print("Response:")
    print(text)

    # Strip text to include only the SQL code block
    text = sanitize_output(text)
    print("Response stripped:")
    print(text)

    return text

#### Definir função auxiliar para executar SQL

A seguinte função auxiliar `execute_sql()` é usada para executar uma consulta SQL no dataset ativo do BigQuery e retornar resultados como um dataframe.

Observe como `execute_sql()` garante a qualificação dos nomes das tabelas com o projeto e o dataset do BigQuery especificado acima, antes de executar a consulta SQL.

In [ ]:
# Limit number of bytes processed as a guardrail for cost control
BQ_MAX_BYTES_BILLED = pow(2, 30)  # 1GB


def execute_sql(query: str):
    print("Executing SQL...")

    # Qualify table names with your project and dataset ID
    for table_name in BQ_TABLES:
        query = query.replace(
            table_name, f"{BQ_PROJECT_ID}.{BQ_LINKED_DATASET}.{table_name}"
        )

    print("Query:")
    print(query)

    # Validate the query by performing a dry run without incurring a charge
    job_config = bigquery.QueryJobConfig(use_query_cache=False, dry_run=True)
    try:
        response = client.query(query, job_config=job_config)
    except Exception as e:
        print("Error validating query:")
        print(e)
        return e

    print("Query will process {:.2f} KB.".format(response.total_bytes_processed / 1024))

    # Execute the query
    job_config = bigquery.QueryJobConfig(
        use_query_cache=False, maximum_bytes_billed=BQ_MAX_BYTES_BILLED
    )
    try:
        response = client.query(query)
        df = response.to_dataframe()
    except Exception as e:
        print("Error executing query:")
        print(e)
        return e

    return df

### Vamos là

Vamos gerar o SQL para responder a este exemplo de pergunta:

In [ ]:
question = "  Escreva a consulta GoogleSQL para a seguinte pergunta: {user_question}"
answer = "  Resposta: \"Query here\""

user_question = "Qual é a meia do potencial do real madrid?"

question = question.format(
    user_question=user_question
)

contents = [
    instructions,
    schema_prompt,
    few_examples,
    question,
    answer,
]

query = generate_sql(
    model,
    contents
    )

print(query)

Vamos testar a consulta gerada com o dataset ativo no BigQuery.

In [ ]:
# Execute the query
query_result = execute_sql(query)
display(query_result)

## Querying Imagens

### Agora vamos usar Gemini 1.0 Pro Vision

In [ ]:
multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#### Definir funções auxiliares para imagens

In [ ]:
import http.client
import typing
import urllib.request

import IPython.display
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps


def display_images(
    images: typing.Iterable[Image],
    max_width: int = 600,
    max_height: int = 350,
) -> None:
    for image in images:
        pil_image = typing.cast(PIL_Image.Image, image._pil_image)
        if pil_image.mode != "RGB":
            # RGB is supported by all Jupyter environments (e.g. RGBA is not yet)
            pil_image = pil_image.convert("RGB")
        image_width, image_height = pil_image.size
        if max_width < image_width or max_height < image_height:
            # Resize to display a smaller notebook image
            pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))
        IPython.display.display(pil_image)


def get_image_bytes_from_url(image_url: str) -> bytes:
    with urllib.request.urlopen(image_url) as response:
        response = typing.cast(http.client.HTTPResponse, response)
        image_bytes = response.read()
    return image_bytes


def load_image_from_url(image_url: str) -> Image:
    image_bytes = get_image_bytes_from_url(image_url)
    return Image.from_bytes(image_bytes)


def display_content_as_image(content: str | Image | Part) -> bool:
    if not isinstance(content, Image):
        return False
    display_images([content])
    return True


def display_content_as_video(content: str | Image | Part) -> bool:
    if not isinstance(content, Part):
        return False
    part = typing.cast(Part, content)
    file_path = part.file_data.file_uri.removeprefix("gs://")
    video_url = f"https://storage.googleapis.com/{file_path}"
    IPython.display.display(IPython.display.Video(video_url, width=600))
    return True


def print_multimodal_prompt(contents: list[str | Image | Part]):
    """
    Given contents that would be sent to Gemini,
    output the full multimodal prompt for ease of readability.
    """
    for content in contents:
        if display_content_as_image(content):
            continue
        if display_content_as_video(content):
            continue
        print(content)

#### Vamos entender uma imagem

O CR Ganondorf não existe no dataset do FIFA 23, porém podemos ver o potencial de cada jogador diretamente no videogame, vamos obter a media do potencial to time usando a imagem com essa informação:

![CR Ganondorf](https://storage.googleapis.com/sre-demos-files/cr_ganondorf_team.jpeg)

In [ ]:
cr_ganondorf_url = "https://storage.googleapis.com/sre-demos-files/cr_ganondorf_team.jpeg"
cr_ganondorf_image = load_image_from_url(cr_ganondorf_url)

image_instructions = """
  Instruções: Considere a seguinte imagem que contém jogadores de futebol:
"""

prompt1 = """
  Cada área dourada representa a informação de cada jogador
  O potencial é o numero que esta na parte esquerda do rosto do jogador.
  A posição é as letras embaixo do potencial.

  Mostra numa tabela o numero de ordem, o potencial e a posição.

  Qual é a meia do potencial do time?
"""


contents = [
    image_instructions,
    cr_ganondorf_image,
    prompt1,
]

responses = multimodal_model.generate_content(contents)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Response--------")
Markdown(responses.text)

Testing Videos prompt


In [ ]:
prompt = """
Responde as perguntas o mais preciso possivel usando o contexto do video.
Se a resposta nao estiver no contexto, responde "resposta nao disponivel no contexto"

Contexto:
O video é um jogo de futebol entre os times

O time UNI usa camisa branca, calção e meiões brancos.
O time ALI usa Camisa azul marinho com detalhes em branco, calção e meiões azuis marinho.

Descreve os goals feitos no video

"""
video = Part.from_uri(
    uri="gs://sre-demos-files/lima.mp4",
    mime_type="video/mp4",
)
contents = [prompt, video]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Response--------")
for response in responses:
    print(response.text, end="")